In [1]:
import pandas as pd
import yfinance as yf
import datetime as dt
from tqdm import tqdm
import time

In [3]:
# Load S&P 500 constituents data
sp500_df = pd.read_csv('sp500_ticker_start_end.csv', parse_dates=['start_date', 'end_date'])

In [ ]:
# Lists to store results
top_volatile_all_years = []
bottom_volatile_all_years = []

# Loop over years
for year in range(2005, 2025):
    print(f"\n\n===== Processing year: {year} =====")
    year_start = dt.datetime(year, 1, 1)
    year_end = dt.datetime(year, 12, 31)

    # Stocks active in the given year
    in_index = sp500_df[
        (sp500_df['start_date'] <= year_start) &
        ((sp500_df['end_date'].isna()) | (sp500_df['end_date'] >= year_end))
    ]['ticker'].unique().tolist()

    print(f"Number of stocks in the index in {year}: {len(in_index)}")

    def download_adjclose_single(ticker, start, end):
        try:
            df = yf.download(ticker, start=start, end=end, progress=False, auto_adjust=False)
            if df.empty or 'Adj Close' not in df.columns:
                return None
            adj_close = df[['Adj Close']].copy()
            adj_close.columns = [ticker]
            return adj_close
        except Exception as e:
            print(f"Error downloading {ticker}: {e}")
            return None

    all_adj_close = []

    for i, ticker in enumerate(in_index):
        print(f"Downloading data: {i+1}/{len(in_index)} — {ticker}")
        data = download_adjclose_single(ticker, year_start, year_end)
        if data is not None:
            all_adj_close.append(data)
        time.sleep(0.1)  # Pause for safety

    if not all_adj_close:
        print(f"No data available for year {year}")
        continue

    data = pd.concat(all_adj_close, axis=1)
    returns = data.pct_change().abs()

    # Remove stocks with more than 10% missing data
    returns = returns.loc[:, returns.isna().mean() < 0.1]

    # Compute volatility as sum of daily absolute returns
    volatility = returns.sum()

    vol_df = pd.DataFrame({
        'ticker': volatility.index,
        'annual_volatility': volatility.values,
        'year': year
    })

    # Append to cumulative lists
    top_volatile = vol_df.sort_values('annual_volatility', ascending=False).head(50)
    bottom_volatile = vol_df.sort_values('annual_volatility', ascending=True).head(50)

    top_volatile_all_years.append(top_volatile)
    bottom_volatile_all_years.append(bottom_volatile)

# Combine results into final tables
top_combined = pd.concat(top_volatile_all_years).sort_values(['year', 'annual_volatility'], ascending=[True, False])
bottom_combined = pd.concat(bottom_volatile_all_years).sort_values(['year', 'annual_volatility'], ascending=[True, True])

In [ ]:
print("\n===== Top 50 most volatile stocks per year =====")
print(top_combined.head(10))

print("\n===== Top 50 least volatile stocks per year =====")
print(bottom_combined.head(10))

# Save to CSV
top_combined.to_csv("Projects/Project_3/top_50_volatile_per_year.csv", index=False)
bottom_combined.to_csv("Projects/Project_3/bottom_50_volatile_per_year.csv", index=False)

In [ ]:
df = pd.read_csv('Projects/Project_3/top_50_volatile_per_year.csv')

# Initialize columns
df['1Y_return'] = None
df['1Y_monthly_investment_return'] = None
df['price_source'] = 'Adj Close'

# Function to download data and compute returns
def compute_returns_for_row(row):
    ticker = row['ticker']
    year = int(row['year'])
    start = dt.datetime(year, 1, 1)
    end = dt.datetime(year, 12, 31)

    try:
        # Download data from Yahoo Finance
        df_yf = yf.download(ticker, start=start, end=end, progress=False, auto_adjust=False)
        if df_yf.empty or 'Adj Close' not in df_yf.columns:
            raise ValueError("No data or missing 'Adj Close' column")

        price_series = df_yf['Adj Close'].dropna()
        if price_series.empty:
            raise ValueError("Empty price series")

        # Annual return (first and last trading day of the year)
        start_price = float(price_series.iloc[0])
        end_price = float(price_series.iloc[-1])
        return_1y = (end_price - start_price) / start_price

        # Monthly investments: first available day >= first business day of each month
        monthly_dates = pd.date_range(start=start, end=end, freq='BMS')
        monthly_prices = []
        for date in monthly_dates:
            possible_dates = price_series.loc[date:].index
            if len(possible_dates) == 0:
                continue
            actual_date = possible_dates[0]
            price = float(price_series.loc[actual_date])
            monthly_prices.append(price)

        if len(monthly_prices) < 2:
            raise ValueError("Not enough monthly data to compute return")

        # Monthly investment of 1 currency unit
        shares_bought = [1.0 / price for price in monthly_prices]
        total_units = sum(shares_bought)
        total_cost = len(monthly_prices)
        final_price = monthly_prices[-1]
        final_value = total_units * final_price
        monthly_return = (final_value - total_cost) / total_cost

        return float(round(return_1y, 6)), float(round(monthly_return, 6))

    except Exception as e:
        print(f"Error for {ticker} in {year}: {e}")
        return float('nan'), float('nan')

# Apply to all rows
tqdm.pandas()
df[['1Y_return', '1Y_monthly_investment_return']] = df.progress_apply(
    compute_returns_for_row, axis=1, result_type='expand'
)

# Save results
df.to_csv('Projects/Project_3/top_50_volatile_with_returns.csv', index=False)
print("File saved as: top_50_volatile_with_returns.csv")

In [ ]:
# Repeat for bottom 50 least volatile stocks
df = pd.read_csv('bottom_50_volatile_per_year.csv')

# Initialize columns
df['1Y_return'] = None
df['1Y_monthly_investment_return'] = None
df['price_source'] = 'Adj Close'

# Compute returns function (same logic)
def compute_returns_for_row(row):
    ticker = row['ticker']
    year = int(row['year'])
    start = dt.datetime(year, 1, 1)
    end = dt.datetime(year, 12, 31)

    try:
        df_yf = yf.download(ticker, start=start, end=end, progress=False, auto_adjust=False)
        if df_yf.empty or 'Adj Close' not in df_yf.columns:
            raise ValueError("No data or 'Adj Close' column missing")

        price_series = df_yf['Adj Close'].dropna()
        if price_series.empty:
            raise ValueError("Price series is empty")

        # Annual return
        start_price = float(price_series.iloc[0])
        end_price = float(price_series.iloc[-1])
        return_1y = (end_price - start_price) / start_price

        # Monthly investment dates
        monthly_dates = pd.date_range(start=start, end=end, freq='BMS')
        monthly_prices = []
        for date in monthly_dates:
            possible_dates = price_series.loc[date:].index
            if len(possible_dates) == 0:
                continue
            actual_date = possible_dates[0]
            price = float(price_series.loc[actual_date])
            monthly_prices.append(price)

        if len(monthly_prices) < 2:
            raise ValueError("Not enough monthly data to compute return")

        shares_bought = [1.0 / price for price in monthly_prices]
        total_units = sum(shares_bought)
        total_cost = len(monthly_prices)
        final_price = monthly_prices[-1]
        final_value = total_units * final_price
        monthly_return = (final_value - total_cost) / total_cost

        return float(round(return_1y, 6)), float(round(monthly_return, 6))

    except Exception as e:
        print(f"Error for {ticker} in {year}: {e}")
        return float('nan'), float('nan')

# Apply to all rows
tqdm.pandas()
df[['1Y_return', '1Y_monthly_investment_return']] = df.progress_apply(
    compute_returns_for_row, axis=1, result_type='expand'
)

# Save results
df.to_csv('bottom_50_volatile_with_returns_final.csv', index=False)
print("Results saved to: bottom_50_volatile_with_returns_final.csv")